In [1]:
# !pip install earthengine-api
# !pip install geehydro
# !pip install geemap
# !pip install geetools
# !pip install jupyter_contrib_nbextensions

In [240]:
import ee
import folium
import geehydro
import geemap
import geetools
import os 

In [3]:
# ee.Authenticate()
ee.Initialize()

# Obtaining NDVI Data using Landsat 5-8

### Defining necessary functions

In [241]:
#cloud mask
from geetools import cloud_mask
mask_LS_SR = cloud_mask.landsatSR()

#ndvi function for image collections for landsat 8
def ndviFun_ls8(im):
  ndvi = im.normalizedDifference(['B5','B4']).rename('ndvi').copyProperties(im, ['system:time_start'])
  return im.addBands(ndvi)

#ndvi function for landsat 5 and 7 image collections
def ndviFun_ls57(im):
  ndvi = im.normalizedDifference(['B4','B3']).rename('ndvi').copyProperties(im, ['system:time_start'])
  return im.addBands(ndvi)

#Sensor calibaration functions
# fixed the bug that after arithmatic operation in the function, the output should casted into same type - in this case
# it is toFloat()

#landsat 8 to 7 using coefficient found in D.P.Roy et el (2016)
def applyCoefficientL7(image):
  image_adjusted = image.select('ndvi').toFloat().copyProperties(image, ['system:time_start'])
  return(image_adjusted)

def applyCoefficientL8(image):
  image_adjusted = image.select('ndvi').multiply(0.9589).toFloat().copyProperties(image, ['system:time_start'])
  return(image_adjusted)

#landsat 5 to landsat 7 using coefficient found in Junchang and Masek (2017)
def applyCoefficientL5(image):
  image_adjusted = image.select('ndvi').multiply(1.037).toFloat().copyProperties(image, ['system:time_start'])
  return(image_adjusted)

### Soum Boundary and Landcover (30m) Mask

In [244]:
# search what files are in the GEE asset folder
# geemap.ee_search()

In [245]:
# Read soum shapefiles from GEE assets
soums = ee.FeatureCollection('users/ta346/soum')

In [246]:
Map = geemap.Map()
Map.centerObject(soums)
Map.addLayer(soums, {}, 'Soums')
Map

Map(center=[46.966158358879866, 103.14166740459963], controls=(WidgetControl(options=['position', 'transparent…

In [209]:
# set out_directory
out_dir = r'U:\honors thesis\gee_python\downloads'
# filename = os.path.join(out_dir, 'soum_new.shp')

In [9]:
# For the recod: this is nice function from geemap to create download link for GEE object
# geemap.ee_export_vector(soums, filename=filename, keep_zip=True)
# geemap.create_download_link(filename)

### Landcover 30m

In [247]:
#2020 LANDCOVER 30m
# //Source: http://www.globallandcover.com/defaults_en.html?src=/Scripts/map/defaults/En/download_en.html&head=download&type=data
# //name: 2020 global landcover 30m
# //downloaded locally and imported as an asset

# //Classification codes: Cultivated Land：10；Forest：20；Grass Land：30；Shrubland：40；Wetland：50；Water Body：60；Tundra：70；Artificial Surfaces：80；Bareland：90；Permanent Snow and Ice：100
# //mask: wetland 50, waterbody 60, Artificial surfaces 80, Permenant snow and ice 100
landcover = ee.Image('users/ta346/2020LC30')
lc = landcover.clip(soums).select('b1')

#to make landcover mask and display - masked areas are black (i.e zero)
mask_lc = lc.neq(50).And(lc.neq(60)).And(lc.neq(80)).And(lc.neq(100))
Map.addLayer(mask_lc, {}, 'landcover (30m)') #Looks Ok!

#function to map over the ndvi dataset collection
def maskLandCover (img):
    return img.updateMask(mask_lc).copyProperties(img, ['system:time_start'])

# Preliminary NDVI dataset (not adjusted by distinct dates)

In [211]:
# create collection images of landsat 5, 7, and 8; calibrated, filtered for 6, 7, 8 months.
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(soums) \
    .filter(ee.Filter.dayOfYear(120, 240)) \
    .filterDate('2013-05-01', '2020-08-31') \
    .map(mask_LS_SR) \
    .map(maskLandCover) \
    .map(ndviFun_ls8) \
    .map(applyCoefficientL8)
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .filterBounds(soums) \
    .filter(ee.Filter.dayOfYear(120, 240)) \
    .filterDate('1999-05-01', '2020-08-31') \
    .map(mask_LS_SR)  \
    .map(maskLandCover) \
    .map(ndviFun_ls57) \
    .map(applyCoefficientL7) 
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filterBounds(soums) \
    .filter(ee.Filter.dayOfYear(120, 240)) \
    .filterDate('1984-05-01', '2011-08-31') \
    .map(mask_LS_SR) \
    .map(maskLandCover) \
    .map(ndviFun_ls57) \
    .map(applyCoefficientL5)

# merge all the collections
mc = ee.ImageCollection(l8.merge(l7.merge(l5))).select('ndvi')

# Downloading single year NDVI dataset by soum and distinct date

In [198]:
#Download ndvi the collection by distinct date and each soum - newcol 
#first, set the dates
start = ee.Date('2020-05-01')
finish = ee.Date('2020-08-31')

# Difference in days between start and finish
diff = finish.difference(start, 'day')
print(diff.getInfo())

temporalResolution = 1  # days - can be adjusted so that collect image collection by 8 days, 16 days, etc

# Make a list of all dates
def function(day):
    return start.advance(day,'day')

# date range
range_dates = ee.List.sequence(0, diff.subtract(1), temporalResolution).map(function)
print(range_dates.size().getInfo())

# for each distinct date in temoralresoultion, mosaic and return as a list of images
def day_mosaics(date, newlist):
  # Cast
  date = ee.Date(date)
  newlist = ee.List(newlist)

  # Filter collection between date and the next day
  filtered = mc.filterDate(date, date.advance(temporalResolution, 'day'))

  # Make the mosaic
  image = ee.Image(filtered.mosaic().clip(soums)) \
                .set("system:index", date.format("YYYMMdd")) 

  # Add the mosaic to a list only if the collection has images
  return ee.List(ee.Algorithms.If(filtered.size(), newlist.add(image), newlist))

#Create image collection from the list of images
newcol = ee.ImageCollection(ee.List(range_dates.iterate(day_mosaics, ee.List([]))))

#Create local projection variable for later to use it for output
landsat_projection = newcol.first().projection()

#Obtain the mean ndvi by each distinct date and soums and count the number of observation of ndvi in the soum
def func_pjp(image):
 return image.reduceRegions(
     collection=soums.select(['au2_code']),
     reducer=ee.Reducer.mean().combine(
      reducer2=ee.Reducer.count(),
      sharedInputs=True),
     scale=30,
     crs = landsat_projection
 ).filter(ee.Filter.neq('mean', None))

output = newcol.map(func_pjp).flatten()

#drop unnecessary geometry information
out = output.select(['.*'], None, False) #FeatureCollection

task = ee.batch.Export.table.toDrive(collection = out,
                            description= 'daily_ndvi_by_soum_2029',
                            folder = 'GEE_ANNUAL_NDVI',
                            fileFormat = 'CSV')
task.start()

122
122


In [14]:
#set dir and file name
out_dir = r'U:\honors thesis\gee_python\downloads'
filename = os.path.join(out_dir, 'monthly_soum_ndvi_2019.csv')

In [15]:
#This is handy function to download FeatureCollection to local folder directly. But it seems that it has limited memory capacity for larger files. 
#download ee.FeatureCollection as a CSV file
# geemap.ee_to_csv(out, filename = filename)

# Downloading NDVI Dataset By Each Year and Distinct Date

In [82]:
# !!!!!!!!!!!!!!This function has bugs - it iterates over only one year
def dailyNDVIComposite():
    startDate = 2000
    lastDate = 2000
    
    for loopYear in range(startDate, lastDate+1, 1):
        
        start = ee.Date.fromYMD(loopYear, 5, 1)
        finish = ee.Date.fromYMD(loopYear, 8, 31)
        
        # Difference in days between start and finish
        diff = finish.difference(start, 'day')
        print(diff.getInfo())
        
        # days - can be adjusted so that collect image collection by 8 days, 16 days, etc
        temporalResolution = 1  

        # date range
        range_dates = ee.List.sequence(0, diff.subtract(1), temporalResolution).map(function)
        print(range_dates.size().getInfo())
        
        # for each distinct date in temoralresoultion, mosaic and return as a list of images
        def day_mosaics(date, newlist):
          # Cast
          date = ee.Date(date)
          newlist = ee.List(newlist)

          # Filter collection between date and the next day
          filtered = mc.filterDate(date, date.advance(temporalResolution, 'day'))

          # Make the mosaic
          image = ee.Image(filtered.mosaic().clip(soums)) \
                        .set("system:index", date.format("YYYMMdd"))  

          # Add the mosaic to a list only if the collection has images
          return ee.List(ee.Algorithms.If(filtered.size(), newlist.add(image), newlist))

        #Create image collection from the list of images using day mosaics function
        newcol = ee.ImageCollection(ee.List(range_dates.iterate(day_mosaics, ee.List([]))))
        
        #Create local projection variable for later to use it for output
        landsat_projection = newcol.first().projection()
        
        #Obtain the mean ndvi by each distinct date and soums and count the number of observation of ndvi in the soum
        def func_pjp(image):
         return image.reduceRegions(
             collection=soums.select(['au2_code']),
             reducer=ee.Reducer.mean().combine(
              reducer2=ee.Reducer.count(),
              sharedInputs=True),
             scale=30,
             crs = landsat_projection
         ).filter(ee.Filter.neq('mean', None))

        #create ee.Feature of mean and count by each soum using func_pjp
        output = newcol.map(func_pjp).flatten()
        
        #drop unnecessary geometry information
        out = output.select(['.*'], None, False) #FeatureCollection
        
        #give name to final output file
        filename = 'daily_ndvi_by_soum_' + str(loopYear)
        
        task = ee.batch.Export.table.toDrive(collection = out,
                                    description= filename,
                                    folder = 'GEE_output',
                                    fileFormat = 'CSV')
        task.start()

In [ ]:
comp = dailyNDVIComposite()

In [ ]:
#check the files that being downloaded
!earthengine task list

# Some More Visualization and Graphs 

In [234]:
# ndvi color function to visualize later
colorizedVis = {
  'min': 0.0,
  'max': 1.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}

#CREATE IMAGE COLLECTION FOR ONLY SUMMER MONTHS BY YEARS

#first, set the dates
start = ee.Date('2020-06-01')
finish = ee.Date('2020-08-31')

# Difference in months between start and finish
diff = finish.difference(start, 'month')
print(diff.getInfo())

temporalResolution = 3  # days or months - can be adjusted so that collect image collection by 8 days, 16 days, etc

# Make a list of all dates
def function(month):
    return start.advance(month,'month')

# date range
range_dates = ee.List.sequence(0, diff.subtract(1), temporalResolution).map(function)
print(range_dates.size().getInfo())

# for each distinct date in temoralresoultion, mosaic and return as a list of images
def month_mean(date, newlist):
  # Cast
  date = ee.Date(date)
  newlist = ee.List(newlist)

  # Filter collection between date and the next day
  filtered = mc.filterDate(date, date.advance(temporalResolution, 'month'))

  # Make the mosaic
  image = ee.Image(filtered.reduce(ee.Reducer.mean()).clip(soums)) \
                .set("system:index", date.format("YYYMMdd")) 

  # Add the mosaic to a list only if the collection has images
  return ee.List(ee.Algorithms.If(filtered.size(), newlist.add(image), newlist))

#Create image collection from the list of images
summer_col = ee.ImageCollection(ee.List(range_dates.iterate(month_mean, ee.List([]))))
# #Create local projection variable for later to use it for output
# landsat_projection = newcol.first().projection()

# #Obtain the mean ndvi by each distinct date and soums and count the number of observation of ndvi in the soum
# def func_pjp(image):
#  return image.reduceRegions(
#      collection=soums.select(['au2_code']),
#      reducer=ee.Reducer.mean().combine(
#       reducer2=ee.Reducer.count(),
#       sharedInputs=True),
#      scale=30,
#      crs = landsat_projection
#  ).filter(ee.Filter.neq('mean', None))

# output = newcol.map(func_pjp).flatten()

# #drop unnecessary geometry information
# out = output.select(['.*'], None, False) #FeatureCollection

# task = ee.batch.Export.table.toDrive(collection = out,
#                             description= 'daily_ndvi_by_soum_2029',
#                             folder = 'GEE_ANNUAL_NDVI',
#                             fileFormat = 'CSV')
# task.start()

2.9897944516314503
1


In [235]:
m3 = geemap.Map()
m3.centerObject(soums)
m3.addLayer(summer_col, colorizedVis, '2020_summer_mean_NDVI')
m3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [228]:
# help(ee.batch.Export.table.toAsset)

In [223]:
bands = ['ndvi']
scale = 30
date_pattern = 'ddMMMy' # dd: day, MMM: month (JAN), y: year
folder = 'MYFOLDER'
data_type = 'uint32'
region = soums

In [227]:
tasks = geetools.batch.Export.imagecollection.toAsset(
            collection=summer_col,
            assetPath = 'summer_col_1985_2020',
            folder=folder,
            region=region,
            scale=scale,
            dataType=data_type,
            datePattern=date_pattern,
            verbose=True,
            maxPixels=int(1e13)
        )

EEException: User memory limit exceeded.

In [ ]:
start.tasks()